In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [7]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4,rnd=233)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.65353, saving model to best_m.h5
 - 3s - loss: 0.6999 - acc: 0.5372 - val_loss: 0.6535 - val_acc: 0.5486
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.6066 - acc: 0.6389 - val_loss: 0.6572 - val_acc: 0.6135
Epoch 3/100
Epoch 00003: val_loss improved from 0.65353 to 0.38137, saving model to best_m.h5
 - 2s - loss: 0.5282 - acc: 0.7372 - val_loss: 0.3814 - val_acc: 0.8204
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.5282 - acc: 0.7441 - val_loss: 0.4078 - val_acc: 0.8005
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.4728 - acc: 0.7716 - val_loss: 0.4215 - val_acc: 0.8030
Epoch 6/100
Epoch 00006: val_loss improved from 0.38137 to 0.30442, saving model to best_m.h5
 - 2s - loss: 0.4385 - acc: 0.7850 - val_loss: 0.3044 - val_acc: 0.8504
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.4478 - acc: 0.7797 - val_loss: 0.3495 - val_acc: 0.8404
Epoch 8/100
Epoch 

Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2255 - acc: 0.9083 - val_loss: 0.2206 - val_acc: 0.9077
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2230 - acc: 0.9100 - val_loss: 0.2166 - val_acc: 0.9127
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.2422 - acc: 0.9000 - val_loss: 0.2166 - val_acc: 0.9252
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.2228 - acc: 0.9167 - val_loss: 0.2077 - val_acc: 0.9102
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2445 - acc: 0.8956 - val_loss: 0.2134 - val_acc: 0.9077
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2185 - acc: 0.9075 - val_loss: 0.2176 - val_acc: 0.9052
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2379 - acc: 0.8983 - val_loss: 0.2084 - val_acc: 0.9252
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.2337 - acc: 0.9050 - val_loss: 0.2080 - val_acc: 0.9127
Epoch 71/100
Epo

Epoch 27/100
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2472 - acc: 0.8942 - val_loss: 0.2632 - val_acc: 0.9002
Epoch 28/100
Epoch 00028: val_loss improved from 0.25312 to 0.24098, saving model to best_m.h5
 - 2s - loss: 0.2463 - acc: 0.8950 - val_loss: 0.2410 - val_acc: 0.9102
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2551 - acc: 0.8875 - val_loss: 0.2691 - val_acc: 0.9077
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2687 - acc: 0.8823 - val_loss: 0.2677 - val_acc: 0.9127
Epoch 31/100
Epoch 00031: val_loss improved from 0.24098 to 0.22609, saving model to best_m.h5
 - 2s - loss: 0.2477 - acc: 0.8989 - val_loss: 0.2261 - val_acc: 0.9202
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.2172 - acc: 0.9042 - val_loss: 0.2265 - val_acc: 0.9152
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2618 - acc: 0.8972 - val_loss: 0.2305 - val_acc: 0.9202
Epoch 34/100
Epoch 00034: val_loss did not improve


Epoch 93/100
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.1809 - acc: 0.9308 - val_loss: 0.2205 - val_acc: 0.9202
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.2250 - acc: 0.9058 - val_loss: 0.2288 - val_acc: 0.9152
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.2012 - acc: 0.9125 - val_loss: 0.2205 - val_acc: 0.9202
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1861 - acc: 0.9275 - val_loss: 0.2261 - val_acc: 0.9177
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.2013 - acc: 0.9125 - val_loss: 0.2270 - val_acc: 0.9127
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.1932 - acc: 0.9139 - val_loss: 0.2230 - val_acc: 0.9227
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.1993 - acc: 0.9192 - val_loss: 0.2737 - val_acc: 0.9052
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.1991 - acc: 0.9167 - val_loss: 0.2283 - val_acc: 0.9152
Epoch 1/100
Epo

Epoch 52/100
Epoch 00052: val_loss did not improve
 - 2s - loss: 0.2394 - acc: 0.8958 - val_loss: 0.2102 - val_acc: 0.9252
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 2s - loss: 0.2189 - acc: 0.9133 - val_loss: 0.2044 - val_acc: 0.9327
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 2s - loss: 0.2243 - acc: 0.9016 - val_loss: 0.2037 - val_acc: 0.9302
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.2278 - acc: 0.9092 - val_loss: 0.2143 - val_acc: 0.9177
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 2s - loss: 0.2159 - acc: 0.9042 - val_loss: 0.2054 - val_acc: 0.9252
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 2s - loss: 0.2181 - acc: 0.9100 - val_loss: 0.2130 - val_acc: 0.9227
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.2242 - acc: 0.9000 - val_loss: 0.2129 - val_acc: 0.9302
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.2198 - acc: 0.9008 - val_loss: 0.2086 - val_acc: 0.9227
Epoch 60/100
Epo

Epoch 15/100
Epoch 00015: val_loss did not improve
 - 2s - loss: 0.3260 - acc: 0.8575 - val_loss: 0.3248 - val_acc: 0.8579
Epoch 16/100
Epoch 00016: val_loss improved from 0.32161 to 0.29151, saving model to best_m.h5
 - 2s - loss: 0.3025 - acc: 0.8575 - val_loss: 0.2915 - val_acc: 0.8778
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.3285 - acc: 0.8564 - val_loss: 0.3117 - val_acc: 0.8628
Epoch 18/100
Epoch 00018: val_loss improved from 0.29151 to 0.28844, saving model to best_m.h5
 - 2s - loss: 0.2845 - acc: 0.8850 - val_loss: 0.2884 - val_acc: 0.8603
Epoch 19/100
Epoch 00019: val_loss improved from 0.28844 to 0.28136, saving model to best_m.h5
 - 2s - loss: 0.2870 - acc: 0.8814 - val_loss: 0.2814 - val_acc: 0.8678
Epoch 20/100
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2807 - acc: 0.8798 - val_loss: 0.2827 - val_acc: 0.8753
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 2s - loss: 0.2893 - acc: 0.8692 - val_loss: 0.2975 - val_acc: 0.8678
Epoch 2

Epoch 75/100
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.2067 - acc: 0.9100 - val_loss: 0.2337 - val_acc: 0.8928
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.2284 - acc: 0.9089 - val_loss: 0.2270 - val_acc: 0.9102
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.2239 - acc: 0.9100 - val_loss: 0.2220 - val_acc: 0.8978
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.2071 - acc: 0.9148 - val_loss: 0.2263 - val_acc: 0.9052
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.2093 - acc: 0.9023 - val_loss: 0.2359 - val_acc: 0.8978
Epoch 80/100
Epoch 00080: val_loss improved from 0.22092 to 0.21068, saving model to best_m.h5
 - 2s - loss: 0.2109 - acc: 0.9242 - val_loss: 0.2107 - val_acc: 0.8953
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 2s - loss: 0.2151 - acc: 0.9092 - val_loss: 0.2182 - val_acc: 0.9002
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 2s - loss: 0.2123 - acc: 0.9039 - val_los

In [8]:
import pickle
with open('../features/cnn_3_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug1_sub.csv', index=False)
# pre 228
# new 2117

# pre 15161
# rnd=1, 15186
# rnd=233, 15378

0.207383216906
         id  is_iceberg
0  5941774d    0.140809
1  4023181e    0.786152
2  b20200e4    0.182997
3  e7f018bb    0.994746
4  4371c8c3    0.061387
